In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI AutoML tables regression


## Installation

Install the latest (preview) version of Vertex SDK.

In [ ]:
! pip3 install -U google-cloud-aiplatform --user

Install the Google *cloud-storage* library as well.

In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("AUTORUN") and False:
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see [Region support for Vertex AI services](https://cloud.google.com/vertex-ai/docs/general/locations)


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex SDK

Import the Vertex SDK into our Python environment.


In [ ]:
import os
import sys
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### Vertex AI constants

Setup up the following constants for Vertex AI:

- `API_ENDPOINT`: The Vertex AI API service endpoint for dataset, model, job, pipeline and endpoint services.
- `API_PREDICT_ENDPOINT`: The Vertex AI API service endpoint for prediction.
- `PARENT`: The Vertex AI location root path for dataset, model and endpoint resources.


In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML constants

Next, setup constants unique to AutoML Table classification datasets and training:

- Dataset Schemas: Tells the managed dataset service which type of dataset it is.
- Data Labeling (Annotations) Schemas: Tells the managed dataset service how the data is labeled (annotated).
- Dataset Training Schemas: Tells the Vertex AI Pipelines service the task (e.g., classification) to train the model for.


In [ ]:
# Tabular Dataset type
TABLE_SCHEMA = "google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml"
# Tabular Labeling type
IMPORT_SCHEMA_TABLE_CLASSIFICATION = (
    "gs://google-cloud-aiplatform/schema/dataset/ioformat/table_io_format_1.0.0.yaml"
)
# Tabular Training task
TRAINING_TABLE_CLASSIFICATION_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml"

## Clients

The Vertex SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (Vertex).

You will use several clients in this tutorial, so set them all up upfront.

- Dataset Service for managed datasets.
- Model Service for managed models.
- Pipeline Service for training.
- Endpoint Service for deployment.
- Job Service for batch jobs and custom training.
- Prediction Service for serving. *Note*: Prediction has a different service endpoint.


In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

In [ ]:
IMPORT_FILE = "gs://cloud-ml-tables-data/bank-marketing.csv"

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10

*Example output*:
```
Age,Job,MaritalStatus,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,PDays,Previous,POutcome,Deposit
58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,1
44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,1
33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,1
47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,1
33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,1
35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,1
28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,1
42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,1
58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,1
```


## Create a dataset

### [projects.locations.datasets.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/create)

#### Request

In [ ]:
DATA_SCHEMA = TABLE_SCHEMA

metadata = {
    "input_config": {
        "gcs_source": {
            "uri": [IMPORT_FILE],
        }
    }
}

dataset = {
    "display_name": "bank_" + TIMESTAMP,
    "metadata_schema_uri": "gs://" + DATA_SCHEMA,
    "metadata": json_format.ParseDict(metadata, Value()),
}

print(
    MessageToJson(
        aip.CreateDatasetRequest(
            parent=PARENT,
            dataset=dataset,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "bank_20210226015209",
    "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml",
    "metadata": {
      "input_config": {
        "gcs_source": {
          "uri": [
            "gs://cloud-ml-tables-data/bank-marketing.csv"
          ]
        }
      }
    }
  }
}
```


#### Call

In [ ]:
request = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/7748812594797871104",
  "displayName": "bank_20210226015209",
  "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/tabular_1.0.0.yaml",
  "labels": {
    "aiplatform.googleapis.com/dataset_metadata_schema": "TABLE"
  },
  "metadata": {
    "inputConfig": {
      "gcsSource": {
        "uri": [
          "gs://cloud-ml-tables-data/bank-marketing.csv"
        ]
      }
    }
  }
}
```


In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

## Train a model

### [projects.locations.trainingPipelines.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)

#### Request

In [ ]:
TRAINING_SCHEMA = TRAINING_TABLE_CLASSIFICATION_SCHEMA

TRANSFORMATIONS = [
    {"auto": {"column_name": "Age"}},
    {"auto": {"column_name": "Job"}},
    {"auto": {"column_name": "MaritalStatus"}},
    {"auto": {"column_name": "Education"}},
    {"auto": {"column_name": "Default"}},
    {"auto": {"column_name": "Balance"}},
    {"auto": {"column_name": "Housing"}},
    {"auto": {"column_name": "Loan"}},
    {"auto": {"column_name": "Contact"}},
    {"auto": {"column_name": "Day"}},
    {"auto": {"column_name": "Month"}},
    {"auto": {"column_name": "Duration"}},
    {"auto": {"column_name": "Campaign"}},
    {"auto": {"column_name": "PDays"}},
    {"auto": {"column_name": "POutcome"}},
]

task = Value(
    struct_value=Struct(
        fields={
            "disable_early_stopping": Value(bool_value=False),
            "prediction_type": Value(string_value="regression"),
            "target_column": Value(string_value="Deposit"),
            "train_budget_milli_node_hours": Value(number_value=1000),
            "transformations": json_format.ParseDict(TRANSFORMATIONS, Value()),
        }
    )
)

training_pipeline = {
    "display_name": "bank_" + TIMESTAMP,
    "input_data_config": {
        "dataset_id": dataset_short_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    },
    "model_to_upload": {
        "display_name": "flowers_" + TIMESTAMP,
    },
    "training_task_definition": TRAINING_SCHEMA,
    "training_task_inputs": task,
}

print(
    MessageToJson(
        aip.CreateTrainingPipelineRequest(
            parent=PARENT,
            training_pipeline=training_pipeline,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "trainingPipeline": {
    "displayName": "bank_20210226015209",
    "inputDataConfig": {
      "datasetId": "7748812594797871104",
      "fractionSplit": {
        "trainingFraction": 0.8,
        "validationFraction": 0.1,
        "testFraction": 0.1
      }
    },
    "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml",
    "trainingTaskInputs": {
      "transformations": [
        {
          "auto": {
            "column_name": "Age"
          }
        },
        {
          "auto": {
            "column_name": "Job"
          }
        },
        {
          "auto": {
            "column_name": "MaritalStatus"
          }
        },
        {
          "auto": {
            "column_name": "Education"
          }
        },
        {
          "auto": {
            "column_name": "Default"
          }
        },
        {
          "auto": {
            "column_name": "Balance"
          }
        },
        {
          "auto": {
            "column_name": "Housing"
          }
        },
        {
          "auto": {
            "column_name": "Loan"
          }
        },
        {
          "auto": {
            "column_name": "Contact"
          }
        },
        {
          "auto": {
            "column_name": "Day"
          }
        },
        {
          "auto": {
            "column_name": "Month"
          }
        },
        {
          "auto": {
            "column_name": "Duration"
          }
        },
        {
          "auto": {
            "column_name": "Campaign"
          }
        },
        {
          "auto": {
            "column_name": "PDays"
          }
        },
        {
          "auto": {
            "column_name": "POutcome"
          }
        }
      ],
      "prediction_type": "regression",
      "disable_early_stopping": false,
      "train_budget_milli_node_hours": 1000.0,
      "target_column": "Deposit"
    },
    "modelToUpload": {
      "displayName": "flowers_20210226015209"
    }
  }
}
```


#### Call

In [ ]:
request = clients["pipeline"].create_training_pipeline(
    parent=PARENT, training_pipeline=training_pipeline
)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/3147717072369221632",
  "displayName": "bank_20210226015209",
  "inputDataConfig": {
    "datasetId": "7748812594797871104",
    "fractionSplit": {
      "trainingFraction": 0.8,
      "validationFraction": 0.1,
      "testFraction": 0.1
    }
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml",
  "trainingTaskInputs": {
    "targetColumn": "Deposit",
    "trainBudgetMilliNodeHours": "1000",
    "transformations": [
      {
        "auto": {
          "columnName": "Age"
        }
      },
      {
        "auto": {
          "columnName": "Job"
        }
      },
      {
        "auto": {
          "columnName": "MaritalStatus"
        }
      },
      {
        "auto": {
          "columnName": "Education"
        }
      },
      {
        "auto": {
          "columnName": "Default"
        }
      },
      {
        "auto": {
          "columnName": "Balance"
        }
      },
      {
        "auto": {
          "columnName": "Housing"
        }
      },
      {
        "auto": {
          "columnName": "Loan"
        }
      },
      {
        "auto": {
          "columnName": "Contact"
        }
      },
      {
        "auto": {
          "columnName": "Day"
        }
      },
      {
        "auto": {
          "columnName": "Month"
        }
      },
      {
        "auto": {
          "columnName": "Duration"
        }
      },
      {
        "auto": {
          "columnName": "Campaign"
        }
      },
      {
        "auto": {
          "columnName": "PDays"
        }
      },
      {
        "auto": {
          "columnName": "POutcome"
        }
      }
    ],
    "predictionType": "regression"
  },
  "modelToUpload": {
    "displayName": "flowers_20210226015209"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-02-26T01:57:51.364312Z",
  "updateTime": "2021-02-26T01:57:51.364312Z"
}
```


In [ ]:
# The full unique ID for the training pipeline
training_pipeline_id = request.name
# The short numeric ID for the training pipeline
training_pipeline_short_id = training_pipeline_id.split("/")[-1]

print(training_pipeline_id)

### [projects.locations.trainingPipelines.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/get)

#### Call

In [ ]:
request = clients["pipeline"].get_training_pipeline(name=training_pipeline_id)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/3147717072369221632",
  "displayName": "bank_20210226015209",
  "inputDataConfig": {
    "datasetId": "7748812594797871104",
    "fractionSplit": {
      "trainingFraction": 0.8,
      "validationFraction": 0.1,
      "testFraction": 0.1
    }
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml",
  "trainingTaskInputs": {
    "trainBudgetMilliNodeHours": "1000",
    "transformations": [
      {
        "auto": {
          "columnName": "Age"
        }
      },
      {
        "auto": {
          "columnName": "Job"
        }
      },
      {
        "auto": {
          "columnName": "MaritalStatus"
        }
      },
      {
        "auto": {
          "columnName": "Education"
        }
      },
      {
        "auto": {
          "columnName": "Default"
        }
      },
      {
        "auto": {
          "columnName": "Balance"
        }
      },
      {
        "auto": {
          "columnName": "Housing"
        }
      },
      {
        "auto": {
          "columnName": "Loan"
        }
      },
      {
        "auto": {
          "columnName": "Contact"
        }
      },
      {
        "auto": {
          "columnName": "Day"
        }
      },
      {
        "auto": {
          "columnName": "Month"
        }
      },
      {
        "auto": {
          "columnName": "Duration"
        }
      },
      {
        "auto": {
          "columnName": "Campaign"
        }
      },
      {
        "auto": {
          "columnName": "PDays"
        }
      },
      {
        "auto": {
          "columnName": "POutcome"
        }
      }
    ],
    "targetColumn": "Deposit",
    "predictionType": "regression"
  },
  "modelToUpload": {
    "displayName": "flowers_20210226015209"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-02-26T01:57:51.364312Z",
  "updateTime": "2021-02-26T01:57:51.364312Z"
}
```


In [ ]:
while True:
    response = clients["pipeline"].get_training_pipeline(name=training_pipeline_id)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_name = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        model_id = response.model_to_upload.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print(model_id)

## Evaluate the model

### [projects.locations.models.evaluations.list](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

#### Call

In [ ]:
request = clients["model"].list_model_evaluations(parent=model_id)

#### Response

In [ ]:
import json

model_evaluations = [json.loads(MessageToJson(me.__dict__["_pb"])) for me in request]
# The evaluation slice
evaluation_slice = request.model_evaluations[0].name

print(json.dumps(model_evaluations, indent=2))

*Example output*:
```
[
  {
    "name": "projects/116273516712/locations/us-central1/models/3936304403996213248/evaluations/6323797633322037836",
    "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/regression_metrics_1.0.0.yaml",
    "metrics": {
      "rSquared": 0.39799774,
      "meanAbsolutePercentageError": 9.791032,
      "rootMeanSquaredError": 0.24675915,
      "rootMeanSquaredLogError": 0.10022795,
      "meanAbsoluteError": 0.12842195
    },
    "createTime": "2021-02-26T03:39:42.254525Z"
  }
]
```


### [projects.locations.models.evaluations.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models.evaluations/get)

#### Call

In [ ]:
request = clients["model"].get_model_evaluation(name=evaluation_slice)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/3936304403996213248/evaluations/6323797633322037836",
  "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/regression_metrics_1.0.0.yaml",
  "metrics": {
    "meanAbsolutePercentageError": 9.791032,
    "rootMeanSquaredLogError": 0.10022795,
    "rSquared": 0.39799774,
    "meanAbsoluteError": 0.12842195,
    "rootMeanSquaredError": 0.24675915
  },
  "createTime": "2021-02-26T03:39:42.254525Z"
}
```


## Make batch predictions

### Make the batch input file

Let's now make a batch input file, which you store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You will use CSV in this tutorial.ion on.

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 1 > tmp.csv
! gsutil cat $IMPORT_FILE | tail -n 10 >> tmp.csv

! cut -d, -f1-16 tmp.csv > batch.csv

gcs_input_uri = "gs://" + BUCKET_NAME + "/test.csv"

! gsutil cp batch.csv $gcs_input_uri

In [ ]:
! gsutil cat $gcs_input_uri

*Example output*:
```
Age,Job,MaritalStatus,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,PDays,Previous,POutcome
53,management,married,tertiary,no,583,no,no,cellular,17,nov,226,1,184,4,success
34,admin.,single,secondary,no,557,no,no,cellular,17,nov,224,1,-1,0,unknown
23,student,single,tertiary,no,113,no,no,cellular,17,nov,266,1,-1,0,unknown
73,retired,married,secondary,no,2850,no,no,cellular,17,nov,300,1,40,8,failure
25,technician,single,secondary,no,505,no,yes,cellular,17,nov,386,2,-1,0,unknown
51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown
71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown
72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success
57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown
37,entrepreneur,married,secondary,no,2971,no,no,cellular,17,nov,361,2,188,11,other
```


### [projects.locations.batchPredictionJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/create)

#### Request

In [ ]:
batch_prediction_job = {
    "display_name": "bank_" + TIMESTAMP,
    "model": model_id,
    "input_config": {
        "instances_format": "csv",
        "gcs_source": {
            "uris": [gcs_input_uri],
        },
    },
    "output_config": {
        "predictions_format": "csv",
        "gcs_destination": {
            "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/",
        },
    },
    "dedicated_resources": {
        "machine_spec": {
            "machine_type": "n1-standard-2",
            "accelerator_count": 0,
        },
        "starting_replica_count": 1,
        "max_replica_count": 1,
    },
}

print(
    MessageToJson(
        aip.CreateBatchPredictionJobRequest(
            parent=PARENT,
            batch_prediction_job=batch_prediction_job,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "batchPredictionJob": {
    "displayName": "bank_20210226015209",
    "model": "projects/116273516712/locations/us-central1/models/3936304403996213248",
    "inputConfig": {
      "instancesFormat": "csv",
      "gcsSource": {
        "uris": [
          "gs://migration-ucaip-trainingaip-20210226015209/test.csv"
        ]
      }
    },
    "outputConfig": {
      "predictionsFormat": "csv",
      "gcsDestination": {
        "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226015209/batch_output/"
      }
    },
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-2"
      },
      "startingReplicaCount": 1,
      "maxReplicaCount": 1
    }
  }
}
```


#### Call

In [ ]:
request = clients["job"].create_batch_prediction_job(
    parent=PARENT, batch_prediction_job=batch_prediction_job
)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/4417450692310990848",
  "displayName": "bank_20210226015209",
  "model": "projects/116273516712/locations/us-central1/models/3936304403996213248",
  "inputConfig": {
    "instancesFormat": "csv",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210226015209/test.csv"
      ]
    }
  },
  "outputConfig": {
    "predictionsFormat": "csv",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226015209/batch_output/"
    }
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-02-26T09:35:43.113270Z",
  "updateTime": "2021-02-26T09:35:43.113270Z"
}
```


In [ ]:
# The fully qualified ID for the batch job
batch_job_id = request.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### [projects.locations.batchPredictionJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/get)

#### Call

In [ ]:
request = clients["job"].get_batch_prediction_job(name=batch_job_id)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/4417450692310990848",
  "displayName": "bank_20210226015209",
  "model": "projects/116273516712/locations/us-central1/models/3936304403996213248",
  "inputConfig": {
    "instancesFormat": "csv",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210226015209/test.csv"
      ]
    }
  },
  "outputConfig": {
    "predictionsFormat": "csv",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226015209/batch_output/"
    }
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-02-26T09:35:43.113270Z",
  "updateTime": "2021-02-26T09:35:43.113270Z"
}
```


In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    response = clients["job"].get_batch_prediction_job(name=batch_job_id)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", response.state)
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(
            response.output_config.gcs_destination.output_uri_prefix
        )
        ! gsutil ls $folder/prediction*

        ! gsutil cat $folder/prediction*
        break
    time.sleep(60)

*Example output*:
```
gs://migration-ucaip-trainingaip-20210226015209/batch_output/prediction-flowers_20210226015209-2021-02-26T09:35:43.034287Z/predictions_1.csv
Age,Balance,Campaign,Contact,Day,Default,Duration,Education,Housing,Job,Loan,MaritalStatus,Month,PDays,POutcome,Previous,predicted_Deposit
72,5715,5,cellular,17,no,1127,secondary,no,retired,no,married,nov,184,success,3,1.6232702732086182
23,113,1,cellular,17,no,266,tertiary,no,student,no,single,nov,-1,unknown,0,1.3257474899291992
34,557,1,cellular,17,no,224,secondary,no,admin.,no,single,nov,-1,unknown,0,1.0801490545272827
25,505,2,cellular,17,no,386,secondary,no,technician,yes,single,nov,-1,unknown,0,1.2516863346099854
73,2850,1,cellular,17,no,300,secondary,no,retired,no,married,nov,40,failure,8,1.5064295530319214
37,2971,2,cellular,17,no,361,secondary,no,entrepreneur,no,married,nov,188,other,11,1.1924527883529663
57,668,4,telephone,17,no,508,secondary,no,blue-collar,no,married,nov,-1,unknown,0,1.1636843681335449
```


## Make online predictions

### [projects.locations.endpoints.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/create)

#### Request

In [ ]:
endpoint = {"display_name": "bank_" + TIMESTAMP}

print(
    MessageToJson(
        aip.CreateEndpointRequest(
            parent=PARENT,
            endpoint=endpoint,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "endpoint": {
    "displayName": "bank_20210226015209"
  }
}
```


#### Call

In [ ]:
request = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/endpoints/6899338707271155712"
}
```


In [ ]:
# The fully qualified ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### [projects.locations.endpoints.deployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/deployModel)

#### Request

In [ ]:
deployed_model = {
    "model": model_id,
    "display_name": "bank_" + TIMESTAMP,
    "dedicated_resources": {
        "min_replica_count": 1,
        "machine_spec": {"machine_type": "n1-standard-2"},
    },
}

traffic_split = {"0": 100}

print(
    MessageToJson(
        aip.DeployModelRequest(
            endpoint=endpoint_id,
            deployed_model=deployed_model,
            traffic_split=traffic_split,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/6899338707271155712",
  "deployedModel": {
    "model": "projects/116273516712/locations/us-central1/models/3936304403996213248",
    "displayName": "bank_20210226015209",
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-2"
      },
      "minReplicaCount": 1
    }
  },
  "trafficSplit": {
    "0": 100
  }
}
```


#### Call

In [ ]:
request = clients["endpoint"].deploy_model(
    endpoint=endpoint_id, deployed_model=deployed_model, traffic_split=traffic_split
)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "deployedModel": {
    "id": "7646795507926302720"
  }
}
```


In [ ]:
# The  numeric ID for the deploy model
deploy_model_id = result.deployed_model.id

print(deploy_model_id)

### [projects.locations.endpoints.predict](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/predict)

### Prepare data item for online prediction

In [ ]:
INSTANCE = {
    "Age": "58",
    "Job": "managment",
    "MaritalStatus": "married",
    "Education": "teritary",
    "Default": "no",
    "Balance": "2143",
    "Housing": "yes",
    "Loan": "no",
    "Contact": "unknown",
    "Day": "5",
    "Month": "may",
    "Duration": "261",
    "Campaign": "1",
    "PDays": "-1",
    "Previous": 0,
    "POutcome": "unknown",
}

#### Request

In [ ]:
instances_list = [INSTANCE]
instances = [json_format.ParseDict(s, Value()) for s in instances_list]

request = aip.PredictRequest(
    endpoint=endpoint_id,
)
request.instances.append(instances)

print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/6899338707271155712",
  "instances": [
    [
      {
        "Education": "teritary",
        "MaritalStatus": "married",
        "Balance": "2143",
        "Contact": "unknown",
        "Housing": "yes",
        "Previous": 0.0,
        "Loan": "no",
        "Duration": "261",
        "Default": "no",
        "Day": "5",
        "POutcome": "unknown",
        "Age": "58",
        "Month": "may",
        "PDays": "-1",
        "Campaign": "1",
        "Job": "managment"
      }
    ]
  ]
}
```


#### Call

In [ ]:
request = clients["prediction"].predict(endpoint=endpoint_id, instances=instances)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "predictions": [
    {
      "upper_bound": 1.685426712036133,
      "value": 1.007092595100403,
      "lower_bound": 0.06719603389501572
    }
  ],
  "deployedModelId": "7646795507926302720"
}
```


### [projects.locations.endpoints.undeployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/undeployModel)


#### Call


In [ ]:
request = clients["endpoint"].undeploy_model(
    endpoint=endpoint_id, deployed_model_id=deploy_model_id, traffic_split={}
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
delete_dataset = True
delete_model = True
delete_endpoint = True
delete_pipeline = True
delete_batchjob = True
delete_bucket = True

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex AI fully qualified identifier for the model
try:
    if delete_model:
        clients["model"].delete_model(name=model_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex AI fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex AI fully qualified identifier for the training pipeline
try:
    if delete_pipeline:
        clients["pipeline"].delete_training_pipeline(name=training_pipeline_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex AI fully qualified identifier for the batch job
try:
    if delete_batchjob:
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME